# Portfolio Stress Test

<img src="img/H4P1Q2_1.png">

<img src="img/H4P1Q2_2.png">

<img src="img/H4P1Q2_3.png">

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [2]:
df = pd.read_csv("MSFT_AAPL_Log_Returns.csv", header=None)
df.columns = ['date', 'MSFT', 'AAPL']
df = df.set_index('date')
df.head()

,MSFT,AAPL
date,,
9/1/2011,-0.014770,-0.010054
9/2/2011,-0.015767,-0.018356
9/6/2011,-0.011304,0.015043
9/7/2011,0.019026,0.010999
9/8/2011,0.008426,0.000547


In [3]:
alpha = 0.95
k = 10
M = 100
N = 50000
lam = 0.97
prcpl = 1000000
prcpl_msft = 448.77/(448.77+575.11)*prcpl
prcpl_aapl = prcpl - prcpl_msft

### (1)

In [4]:
miu0_msft = df['MSFT'][:M].mean()
#sig0_msft = df['MSFT'][:M].std()
miu0_aapl = df['AAPL'][:M].mean()
#sig0_aapl = df['AAPL'][:M].std()
cov0 = np.cov(df['MSFT'][:M].values, df['AAPL'][:M].values)

In [5]:
miu0 = np.matrix([miu0_msft, miu0_aapl]).T
sig0 = np.matrix(cov0)
sig0, miu0

(matrix([[0.00024404, 0.00012241],
         [0.00012241, 0.00030046]]), matrix([[0.00105511],
         [0.0014894 ]]))

In [6]:
data_msft = df["MSFT"][M:].values
data_aapl = df["AAPL"][M:].values
data = np.matrix([data_msft, data_aapl]).T
data

matrix([[-0.00203183, -0.0045551 ],
        [-0.00919468,  0.00596454],
        [ 0.01291656,  0.01275091],
        ...,
        [-0.00362101, -0.00770608],
        [-0.0074556 ,  0.00094295],
        [ 0.00225989,  0.00592024]])

In [7]:
miu_ewma = miu0
sig_ewma = sig0
for i in range(0, len(data_msft)):
    sig_ewma = lam*sig_ewma + (1-lam)*(data[i].T-miu_ewma)*(data[i]-miu_ewma.T)
    miu_ewma = lam*miu_ewma + (1-lam)*data[i].T

In [8]:
sig_ewma, miu_ewma

(matrix([[1.29896289e-04, 3.30063144e-05],
         [3.30063144e-05, 1.51851733e-04]]), matrix([[0.00125025],
         [0.00106069]]))

### (2)

In [9]:
theta = np.matrix([prcpl_msft, prcpl_aapl]).T

In [10]:
var_ewma = -theta.T*miu_ewma + np.sqrt(theta.T*sig_ewma*theta)*norm.ppf(alpha)
var_ewma.A1[0]

14383.846180380486

In [11]:
var_ewma_kday = np.sqrt(k)*var_ewma
var_ewma_kday.A1[0]

45485.71544351549

In [12]:
captl_change = 3*var_ewma_kday
captl_change.A1[0]

136457.14633054647

### (3)

In [13]:
rho = np.sqrt(sig_ewma[0,1]*sig_ewma[1,0]/(sig_ewma[0,0]*sig_ewma[1,1]))
x2 = miu_ewma[1].A1[0] - 5*np.sqrt(sig_ewma[1,1])
miu_x1 = miu_ewma[0].A1[0] + rho*np.sqrt(sig_ewma[0,0])/np.sqrt(sig_ewma[1,1])*(x2-miu_ewma[1].A1[0])
var_x1 = sig_ewma[0,0]*(1-rho**2)

In [14]:
loss_sim = []
for i in range(0, N):
    x1_sum = 0
    x2_sum = 0
    
    sig = sig_ewma
    miu = miu_ewma
    
    x1 = np.random.normal(miu_x1, np.sqrt(var_x1))
    
    for j in range(1, k+1):
        x1, x2 = np.random.multivariate_normal(miu.A1, sig)
        sig = lam*sig + (1-lam)*(np.matrix([x1,x2]).T-miu)*(np.matrix([x1,x2])-miu.T)
        miu = lam*miu + (1-lam)*np.matrix([x1,x2]).T
        x1_sum += x1
        x2_sum += x2
    loss = -(prcpl_msft*x1_sum + prcpl_aapl*x2_sum)
    loss_sim.append(loss)

In [15]:
ave_kday_loss = np.mean(loss_sim)
kday_var = np.quantile(loss_sim, alpha, interpolation="higher")
ave_kday_loss, kday_var

(-11523.443367901582, 44462.87708335159)

In [16]:
u1 = var_ewma_kday.A1[0]
u2 = captl_change.A1[0]
loss_sim = np.array(loss_sim)
exceed_1 = len(loss_sim[loss_sim>u1])/len(loss_sim)*100
exceed_2 = len(loss_sim[loss_sim>u2])/len(loss_sim)*100
u1, u2, exceed_1, exceed_2

(45485.71544351549, 136457.14633054647, 4.716, 0.002)